In [ ]:
import pandas

# set the appropriate options for pandas
pandas.set_option('display.max_rows', 10)

In [ ]:
# load the dataset into the workspace
data = pandas.read_csv("./datasets/energydata_complete.csv",parse_dates=["date"])

In [ ]:
# let's have a peek into the data
#data.head(12)
data

In [ ]:
import warnings

# turning warnings off
warnings.catch_warnings()

from xgboost import XGBRegressor

In [ ]:
# let's see what we have here?!
data.columns, len(data.columns)

In [ ]:
# select target and features
features = ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
            'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7',
            #'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 
            'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 
            'rv1', 'rv2'
           ]
target = ['Appliances']

In [ ]:
# separate X, from Y
X, Y = data[features], data[target]

# use first 95% observation for training the rest for test
train_margin = int(data.shape[0]*0.80)

x_train, x_test = X[0:train_margin], X[train_margin:]
y_train, y = Y[0:train_margin], Y[train_margin:]

In [ ]:
# visit http://xgboost.readthedocs.io/en/latest/parameter.html for more info on hyper parameter tuning
help(XGBRegressor)
pass

In [ ]:
model = XGBRegressor(n_estimators=100, max_depth=5, nthread=2, 
                     silent=False, objective="reg:linear")

# train the model
model.fit(x_train, y_train, eval_metric="rmse", verbose=True)

In [ ]:
# make the predictions
y_hat = model.predict(x_test)
y

In [ ]:
make_df = pandas.DataFrame
y_and_y_hat = pandas.concat([y.rename(columns={"Appliances": "y"}).reset_index(), 
                             make_df({"y_hat": list(y_hat)})], axis="columns")
y_and_y_hat

In [ ]:
from sklearn.metrics import mean_absolute_error, median_absolute_error

In [ ]:
error_function = mean_absolute_error
error_function(y_and_y_hat["y"], y_and_y_hat["y_hat"])

In [ ]:
# let's see
from bokeh.plotting import figure, output_file, show

In [ ]:
output_file("/tmp/line.html")

plot1 = figure(plot_width=800, plot_height=600, y_axis_label="Appliance Power Consumption(KW/h)", 
               x_axis_label="Observation index", title="y(gree) to y_hat(red) comparison")

x = y_and_y_hat["index"]

# add a line renderer
plot1.line(x, y_and_y_hat["y"], line_width=2, color="green")
plot1.line(x, y_and_y_hat["y_hat"], line_width=2, color="red")

show(plot1)